In [115]:
import pandas as pd
import csv
from datetime import datetime, timedelta
from tqdm import tqdm

In [ ]:
df_installs = pd.read_csv("C:\Future\\for git\Task-for-Junior-Data-Analystt\installs.csv",skipinitialspace=True)
df_installs

In [8]:
df_iap = pd.read_csv("C:\Future\\for git\Task-for-Junior-Data-Analystt\iap.csv",skipinitialspace=True)

In [9]:
df_rv = pd.read_csv("C:\Future\\for git\Task-for-Junior-Data-Analystt\\rv.csv",skipinitialspace=True)

In [ ]:
df_inst_rv_merge = df_installs.merge(df_rv, how='left',on='user_id')
df_inst_iap_merge = df_installs.merge(df_iap, how='left',on='user_id')
df_merge = df_inst_rv_merge.merge(df_inst_iap_merge,how='outer',indicator=True)

df_merge

In [34]:
with open('m_df.csv', 'w', newline='') as file:
    writer = csv.writer(file)

In [35]:
df_merge.to_csv('m_df.csv', sep=',', index=False, encoding='utf-8')


1.251525271319444

In [52]:
def days_passed(install_time, merge_time):
    time_difference = pd.Timestamp(merge_time) - pd.Timestamp(install_time)
    return (time_difference.total_seconds() / 3600)

In [108]:
df_merge.insert(2,column = "day1_revenue", value =0) 
df_merge.insert(3,column = "endofDay_revenue", value =0)

In [119]:
def same_day(date1, date2):
    return date1.date() == date2.date()

# Define a function to calculate the time difference in hours
def time_difference_hours(date1, date2):
    return (date2 - date1).total_seconds() / 3600

# Iterate through the DataFrame using vectorized operations where possible
df_merge['install_time'] = pd.to_datetime(df_merge['install_time'])
df_merge['date'] = pd.to_datetime(df_merge['date'])

# Use tqdm to track progress while iterating through the DataFrame
for index, row in tqdm(df_merge.iterrows(), total=len(df_merge)):
    install_time = row['install_time']
    date = row['date']
    time = row['time']
    purchase_value = row['purchase_value']
    ad_revenue = row['ad_revenue']
    
    if not pd.isna(time):
        if days_passed(install_time, time) <= 24:
            df_merge.at[index, 'day1_revenue'] = purchase_value
        if install_time.date() == pd.to_datetime(time).date():
            df_merge.at[index, 'endofDay_revenue'] = purchase_value
            
    if not pd.isna(date):
        difference_hours = (install_time - date).total_seconds() / 3600
        if install_time.date() == date.date():
            df_merge.at[index, 'endofDay_revenue'] = ad_revenue
            df_merge.at[index, 'day1_revenue'] = ad_revenue

100%|██████████| 4104494/4104494 [09:44<00:00, 7019.27it/s] 


In [ ]:

df_merge

In [121]:
with open('t.csv', 'w', newline='') as file:
    writer = csv.writer(file)

In [122]:
df_merge.to_csv('t.csv', sep=',', index=False, encoding='utf-8')

In [123]:
total_purchase = df_merge['purchase_value'].sum()
unique_users = df_installs['user_id'].nunique()
total_ad_revenue = df_rv['ad_revenue'].sum()
total_1day_revenue = df_merge['day1_revenue'].sum()
total_end_of_1day_revenue = df_merge['endofDay_revenue'].sum()
iap_arpu = total_purchase / unique_users
rv_arpu = total_ad_revenue / unique_users
overall_arpu =(total_purchase +total_ad_revenue)/ unique_users
end_of_1day_arpu = total_end_of_1day_revenue / unique_users
day1_ARPU = total_1day_revenue / unique_users

print('iap_arpu :', iap_arpu)
print('rv_arpu :', rv_arpu)
print('overall_arpu :', overall_arpu )
print('day1_ARPU :', day1_ARPU)
print('end_of_1day_arpu :', end_of_1day_arpu)


iap_arpu : 0.7937312154068215
rv_arpu : 0.45701563835608755
overall_arpu : 1.2507468537629092
day1_ARPU : 0.3455214373145867
end_of_1day_arpu : 0.2765293628219536
